In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

class Clasificadores:
    def __init__(self):
        self.regresion_logistica = LogisticRegression(max_iter=1000)
        self.knn = KNeighborsClassifier()
        self.svm = SVC()
        self.arbol_decision = DecisionTreeClassifier()
        self.bosque_aleatorio = RandomForestClassifier()

    def entrenar_modelo(self, modelo, datos_entrenamiento, etiquetas_entrenamiento):
        try:
            modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)
        except Exception as e:
            print(f"Error al entrenar el modelo {modelo}: {e}")

    def generar_predicciones(self, modelo, datos_prueba):
        try:
            return modelo.predict(datos_prueba)
        except Exception as e:
            print(f"Error al predecir con el modelo {modelo}: {e}")
            return None

    def evaluar_modelo(self, etiquetas_reales, etiquetas_predichas):
        print("\nMatriz de Confusión:")
        print(confusion_matrix(etiquetas_reales, etiquetas_predichas))
        print("\nReporte de Clasificación:")
        print(classification_report(etiquetas_reales, etiquetas_predichas))
        print(f"Precisión: {accuracy_score(etiquetas_reales, etiquetas_predichas):.2f}")

# Definición del bloque principal para ejecutar el script
def main():
    # Lectura de datos desde archivo
    ruta_datos = "datos.xlsx"
    try:
        dataset = pd.read_excel(ruta_datos)
    except FileNotFoundError:
        print("El archivo de datos no se encontró.")
        return

    # Visualización inicial de los datos
    print("Primeras filas del conjunto de datos:")
    print(dataset.head())

    # Separación en características (X) y etiquetas (y)
    caracteristicas = dataset.iloc[:, 2:]  # Columnas que contienen las características
    etiquetas = dataset.iloc[:, 1]         # Columna con las etiquetas

    # División en datos de entrenamiento y prueba
    X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(caracteristicas, etiquetas, test_size=0.2, random_state=42)

    # Instancia de la clase de clasificadores
    clasificadores = Clasificadores()

    # Entrenamiento y evaluación de cada modelo
    for nombre_modelo, modelo in clasificadores.__dict__.items():
        print(f"\nEntrenando el modelo: {nombre_modelo}...")
        clasificadores.entrenar_modelo(modelo, X_entrenamiento, y_entrenamiento)

        print(f"Evaluando el modelo: {nombre_modelo}...")
        predicciones = clasificadores.generar_predicciones(modelo, X_prueba)
        if predicciones is not None:
            clasificadores.evaluar_modelo(y_prueba, predicciones)

    # Comparación visual de la precisión promedio de cada modelo
    resultados_validacion = {}
    for nombre_modelo, modelo in clasificadores.__dict__.items():
        try:
            puntuaciones = cross_val_score(modelo, caracteristicas, etiquetas, cv=5, scoring='accuracy')
            resultados_validacion[nombre_modelo] = puntuaciones.mean()
        except Exception as e:
            print(f"Error durante la validación cruzada del modelo {nombre_modelo}: {e}")

    # Gráfica comparativa de precisión
    nombres_modelos = list(resultados_validacion.keys())
    valores_precision = list(resultados_validacion.values())

    plt.figure(figsize=(10, 6))
    sns.barplot(x=valores_precision, y=nombres_modelos, palette="viridis")
    plt.xlabel("Precisión Promedio (Validación Cruzada)")
    plt.ylabel("Modelo")
    plt.title("Comparación de Precisión entre Modelos")
    plt.show()

# Verificación de ejecución directa
if __name__ == "__main__":
    main()
